Una red no convolucional solo funciona bien si las imágenes que le das son muy parecidas a las que usó en el entrenamiento.
Las redes neuronales son muy buenas con las predicciones siempre y cuando definan características.
Cuando le damos una imagen, la red trabajará con el valor de cada píxel, por lo que con que lo cambiemos un poco ya se rompe.
Una red convolucional trabaja primero con la imagen y luego extrae las características para que no depende solo de la posición o el tamaño de las cosas.
Para crear una covolucional, necesitaremos crear dos nuevas capas intermedias, las de convolución y agrupación. Dichas capas extraerán las características relevantes de la imagen para luego trabajar con el resto de capas normales.

Para crear una capa de convolución, no indicamos el número de neuronas, como en una capa de neuronas, sino el número de núcleos que se van a usar. (Las matrices esas raras para aplicar filtros)
tf.keras.layers.Conv2D(32,(3,3), input_shape=(28, 28, 1))
tf.keras.layers.Conv2D(Número de núcleos para procesar la imagen, tamaño de los núcleos (3x3 casillas), tamaño de imagen de entrada(28x28) y el número de canales (no lo tengo claro porqué 1))

Obtendremos entonces 32 imágenes nuevas con diferentes filtrados. La gracia de todo es que el contenido de los núcleos no se especifican porqué los irá creando la red.

Si usamos fotos a color, dado que están hechas a partir de 3 colores RGB. Se hacen 3 convoluciones por núcleo en vez de 1, por lo que se hacen en este ejemplo 96 convoluciones pero solo 32 resultados.

Todo esto es para neuronas simples, que solo miran pixel por pixel. Toca pasar a las complejas, que miran un poco más grande. Para ello usamos la capa de agrupación. Esta capa reducirá el tamaño de la imagen y resaltar las características más importantes.
Esta vez lo que hará será crear una matriz con los números mayores de los núcleos. Con esto el tamaño se reduce y solo usamos los píxeles más significativos.

In [ ]:
# https://youtu.be/eGDSlW93Bng
import tensorflow as tf
import tensorflow_datasets as tfds

#Descargar set de datos de MNIST (Numeros escritos a mano, etiquetados)
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

#Obtener en variables separadas los datos de entrenamiento (60k) y pruebas (10k)
datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

#Funcion de normalizacion para los datos (Pasar valor de los pixeles de 0-255 a 0-1)
#(Hace que la red aprenda mejor y mas rapido)
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #Aqui se pasa de 0-255 a 0-1
  return imagenes, etiquetas

#Normalizar los datos de entrenamiento con la funcion que hicimos
datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

#Agregar a cache (usar memoria en lugar de disco, entrenamiento mas rapido)
datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

clases = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
#Codigo para mostrar imagenes del set, no es necesario ejecutarlo, solo imprime unos numeros :)
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))

for i, (imagen, etiqueta) in enumerate(datos_entrenamiento.take(25)):
  imagen = imagen.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(imagen, cmap=plt.cm.binary)
  plt.xlabel(clases[etiqueta])

plt.show()  

Creamos el modelo con una capa Flatten de 28 x 28 píxeles y 1 solo canal para blanco y negro.
Agregamos 2 capas ocultas con 50 neuronas con activación ReLu y una capa de salida con softmax. 

In [ ]:
#Crear el modelo (Modelo denso, regular, sin redes convolucionales todavia)
modelo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)), #1 = blanco y negro
    tf.keras.layers.Dense(units=50, activation='relu'),
    tf.keras.layers.Dense(units=50, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

#Compilar el modelo
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

Creamos las variables para los datos de entrenamiento y de test.
Mezclamos y repetimos para que la red no se aprenda el orden.

In [ ]:
#Los numeros de datos de entrenamiento y pruebas (60k y 10k)
num_datos_entrenamiento = metadatos.splits["train"].num_examples
num_datos_pruebas = metadatos.splits["test"].num_examples

#Trabajar por lotes
TAMANO_LOTE=32

#Shuffle y repeat hacen que los datos esten mezclados de manera aleatoria
#para que el entrenamiento no se aprenda las cosas en orden
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_datos_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

Realizamos un entrenamiento de 60 Epochs

In [ ]:
#Realizar el entrenamiento
import math

historial = modelo.fit(
    datos_entrenamiento,
    epochs=60,
    steps_per_epoch=math.ceil(num_datos_entrenamiento/TAMANO_LOTE)
)

Exportar modelo de salida

In [ ]:
#Exportar el modelo al explorador! (Mas detalle de esto en en mi video de exportacion: https://youtu.be/JpE4bYyRADI )
modelo.save('numeros_regular.h5')

#Convertirlo a tensorflow.js
!pip install tensorflowjs

!mkdir carpeta_salida

!tensorflowjs_converter --input_format keras numeros_regular.h5 carpeta_salida